In [2]:
## Helper functions

## Good old Euclidean algorithm
def gen_euc_alg(n, m):
    old_r, r = n, m
    old_s, s = 1, 0
    old_t, t = 0, 1

    while r > 0:
        q = old_r // r

        old_r, r = r, old_r - q * r
        old_s, s = s, old_s - q * s
        old_t, t = t, old_t - q * t

    return old_s, old_t, old_r

def is_prime(n):
    i = 2
    while i**2 <= n:
        if n % i == 0:
            return False
        i += 1
    return True


def base_2(n):
    powers = []

    while n > 0: 
        powers.append(n % 2)
        n = n // 2

    return powers
    
def k_power_mod_n(a, k, n):
    _, _, r = gen_euc_alg(a, n)
    if r != 1:
        print(f'(!) gcd({a}, {n}) = {r}')
        return

    # Expand k in base 2
    k_base_2 = base_2(k)

    # Print k as a power of 2s
    k_str = f'{k} = '
    
    for deg in range(len(k_base_2) - 1):
        if k_base_2[deg]: k_str += f'{2**deg} + '
    k_str += f'{2**(len(k_base_2) - 1)}'
    
    # a_powers will store the successive squares of a: a_powers[i] = a^(2^i)
    a_powers = [a]
    outlist = []
    
    # Start the expansion of the result as a product of a^(2^i)
    if k_base_2[0]: 
        ans = a
        outlist.append((a,a))
    else: ans = 1

    # Compute powers of a by successive squaring, use the base 2 expansion of k to evaluate
    for i in range(1, len(k_base_2)):
        a_powers.append(a_powers[i-1]**2 % n)

        if k_base_2[i]: 
            ans = ans * a_powers[i] % n
            outlist.append((f'{a}^{2**i}', a_powers[i]))

    # Return answer as an integer
    return ans

In [4]:
## ASCII dictionaries & RSA encryption functions
int_to_ASCII = {i: chr(i) for i in range(128)}
ASCII_to_int = {value:key for key, value in int_to_ASCII.items()}

def RSA_encrypt(txt, e, N):
    to_encrypt = []

    # Set max length of word to be encoded at a time
    word_len = len(str(N)) // 3
    
    # Split the text into chunks of text of given word length
    chunks = [txt[i : i + word_len] for i in range(0, len(txt), word_len)]
    
    for chunk in chunks:
        temp_str = ''
        for char in chunk:
            num = str(ASCII_to_int[char])

            # Formatting to deal with leading 0s
            while len(num) < 3: 
                num = '0' + num

            temp_str += num
            
        to_encrypt.append(int(temp_str))

    return [k_power_mod_n(term, k=e, n=N) for term in to_encrypt]

def RSA_decrypt(message, d, N):
    
    decrypted = [k_power_mod_n(term, k=d, n=N) for term in message]
    
    to_str = []
    
    for term in decrypted:
        word = str(term)

        # Deal with leading 0s
        while len(word) % 3 != 0:
            word = '0' + word
        
        chunks = [word[i: i+3] for i in range(0, len(word), 3)]
        to_str.append(chunks)
    
    out_str = ''

    # Create the string of text to return
    for word in to_str:
        for entry in word:
            out_str += int_to_ASCII[int(entry) % 128]

    return out_str

In [4]:
## Create RSA key pair

## Input some kinda big (10 or more digits) prime numbers here, try some really big ones too
## Note: 'true' RSA encryption uses at >2048-bit primes, which have over 600 digits
p = 888888877777777
q = 980835832582657

## Check these are actually primes
#tru_p, tru_q = is_prime(p), is_prime(q)

# if not tru_p:
    # print('p is not prime')

# if not tru_q:
    # print('q is not prime')

## Set the modulus to the produt of p, q
N = p * q

## Since p, q are primes the totient function is easy to compute phi = totient(N). 
## Note: the difficulty of this calculation is where the security of RSA comes from, you don't want
##       to calculate this using some algorithmic process for totient(N)
phi = (p - 1) * (q - 1)

## You pick this 
e = 4398491891113112094317

## e must have gcd(e, phi) = 1, but also have large order mod phi.
## In practice this is something to choose wisely, but for this purpose just guess until you get something which is coprime. 
d, _, r = gen_euc_alg(e, phi)

if r != 1: print(f'e is not coprime with phi')

## Replace e, d mod phi
d = d % phi
e = e % phi

## Double check that we have a valid RSA key
if e * d % phi == 1:
    print('Valid RSA key:\n')
   
    ## Your public key is e, N
    ## A message m gets encoded by x = m^e (mod N)
    print('Public key (e, N) =', (e, N))

    ## Your private key is d, N
    ## Since e * d = 1 (mod phi) an encoded message x gets decoded by m = x^d (mod N)
    print('Private key (d, N) =', (d, N))
    
    ## We'll use ASCII to encode text as integers between 0-127. The number of digits in N
    ## determines the maximum ''chunk'' length of text that can be encoded at once
    print('\nWord length:', len(str(N)) // 3, 'chars')

Valid RSA key:

Public key (e, N) = (4398491891113112094317, 871854062508629541764830213489)
Private key (d, N) = (570790674162356109894530421605, 871854062508629541764830213489)

Word length: 10 chars


In [45]:
import random

in_text = "Cat"

enc_text = RSA_encrypt(in_text, e=e, N=N)

out_text = RSA_decrypt(enc_text, d=d, N=N)

print(f'Using RSA encryption with public key (e, N) = ({e}, {N})')
print()

print('Initial message:', in_text)
print()

print('Encoded message:', enc_text)
print()

print('Unencrypted:', out_text)


## Make sure the encryption worked
if in_text == out_text:
    print('Success!')

else: 
    print('Something went wrong')

print()
print('Attempted decryption with wrong key:', RSA_decrypt(enc_text, d=random.randint(1, N), N=N))



Using RSA encryption with public key (e, N) = (4398491891113112094317, 871854062508629541764830213489)

Initial message: Cat

Encoded message: [302037077332266787106925054424]

Unencrypted: Cat
Success!

Attempted decryption with wrong key: @w+`+A


In [31]:
# Some RSA keys of different size that work

# p = 888888877777777
# q = 980835832582657
# e = 4398491891113112094317

# p = 6899801
# q = 6910481
# e = 4398491891